# Set up

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
return false;
}

In [ ]:
import matplotlib.pyplot as plt
import gstlearn as gl
import gstlearn.plot as gp
import numpy as np
import pandas as pd
import scipy as sp
from scipy import interpolate
import os
import urllib.request

In [ ]:
#!pip install gstlearn
#!pip uninstall -y gstlearn
#!pip3 install numpy --upgrade

# Load Data

In [ ]:
url = 'https://soft.minesparis.psl.eu/gstlearn/data/Alluvial/Oise_GridVectorFinal.ascii'
filename, head = urllib.request.urlretrieve(url)
grid = gl.DbGrid.createFromNF(filename)
grid

In [ ]:
ranges = [500,10000]
oldMethod = False

In [ ]:
url = 'https://soft.minesparis.psl.eu/gstlearn/data/Alluvial/Oise_Data.ascii'
filename, head = urllib.request.urlretrieve(url)
data = gl.Db.createFromNF(filename)
data

# Krigeage SPDE

In [ ]:
myVarioParamBidir = gl.VarioParam()
mydir = gl.DirParam(40,800.,0.5,45.,0,0,np.nan,np.nan,0.,[],[1,1])
myVarioParamBidir.addDir(mydir)
mydir = gl.DirParam(20,400.,0.5,45.,0,0,np.nan,np.nan,0.,[],[-1,1])
myVarioParamBidir.addDir(mydir)
myVarioBidir = gl.Vario(myVarioParamBidir,data)
err = myVarioBidir.compute(gl.ECalcVario.VARIOGRAM)
myVarioBidir.display()
ax = gp.varmod(myVarioBidir,idir=0)
ax.decoration(title="Bi-directional Variogram for thickness(45°)")
ax = gp.varmod(myVarioBidir,idir=1)
ax.decoration(title="Bi-directional Variogram for thickness(45/135°)")

In [ ]:
model = gl.Model.createFromDb(data)


structs = [gl.ECov.NUGGET,gl.ECov.BESSEL_K]

#consNug = gl.ConsItem.define(gl.EConsElem.SILL,0, type = gl.EConsType.UPPER,value = 0.1)

cons1P = gl.ConsItem.define(gl.EConsElem.PARAM,1, type = gl.EConsType.EQUAL,value = 1)
#cons1Rm = gl.ConsItem.define(gl.EConsElem.RANGE,1, type = gl.EConsType.LOWER,value = 100)
#cons1RM = gl.ConsItem.define(gl.EConsElem.RANGE,1, type = gl.EConsType.UPPER,value = 350)

#cons2P = gl.ConsItem.define(gl.EConsElem.PARAM,2, type = gl.EConsType.EQUAL,value = 2)
#cons2Rm = gl.ConsItem.define(gl.EConsElem.RANGE,2, type = gl.EConsType.LOWER,value = 100)
#cons2RM = gl.ConsItem.define(gl.EConsElem.RANGE,2, type = gl.EConsType.UPPER,value = 400)

a = gl.Constraints()
#a.addItem(consNug)
a.addItem(cons1P)
#a.addItem(cons1Rm)
#a.addItem(cons1RM)
#a.addItem(cons2P)
#a.addItem(cons2Rm)
#a.addItem(cons2RM)

err = model.fit(myVarioBidir,structs,constraints=a)


#err = model.fit(myVarioBidir,[gl.ECov.NUGGET,gl.ECov.BESSEL_K,gl.ECov.BESSEL_K], False)

model.display()
ax = gp.varmod(myVarioBidir,model,idir=0)
ax = gp.varmod(myVarioBidir,model,idir=1)

In [ ]:
if not oldMethod :
    model = gl.Model.createFromDb(data)
    covBessel = gl.CovAniso.createAnisotropic(type = gl.ECov.BESSEL_K,ranges=ranges,sill=4.7,param=1,ctxt=model.getContext())
    covNugget = gl.CovAniso.createIsotropic(type = gl.ECov.NUGGET,sill=2.2,ctxt=model.getContext(),range=1)
    model.addCov(covNugget)
    model.addCov(covBessel)


In [ ]:
grid.setLocator("Poly*", gl.ELoc.SEL)
#grid.setLocator("Poly*",gl.ELoc.UNKNOWN)

In [ ]:
model.setDriftIRF(order = 0)
model

In [ ]:
u = grid["Migrate.u_interp"]
v = grid["Migrate.v_interp"]
res = 0. * u
for i in range(u.shape[0]):
    codir = gl.VectorDouble()
    codir.push_back(u[i])
    codir.push_back(v[i])
    res[i] = gl.GH.rotationGetAngles(codir)[0]
    

In [ ]:
grid.deleteColumn("angles*")
if not oldMethod:
    grid["angles1"]= res
    grid.setLocator("angles*",gl.ELoc.NOSTAT)
    nostat = gl.NoStatArray(["M2A"],grid)
    model.addNoStat(nostat)

In [ ]:
grid.deleteColumns(["spde*"])

In [ ]:
spde = gl.SPDE(model,grid,data,gl.ESPDECalcMode.KRIGING)

In [ ]:
uid_result= spde.compute(grid)
grid

In [ ]:
#fig,ax = gp.initGeographic()
#ax.raster(grid,"spde*kriging",cmap="gnuplot2", flagLegend = True)
plt.figure(figsize=[20,12])
gp.raster(grid,"spde.Thickness*estim",cmap="gnuplot2", flagLegend = True)

 # Prepare Export to ArcGIS

In [ ]:
#Kriging data to new grid
#raster = gl.DbGrid()
#raster.reset([2400,2600],[50,50],[625000,6870000],[0,0])
#gl.migrate(grid, raster, "spde.Thickness.kriging",2,[100,100],True)

In [ ]:
# Export Kriging grid to Ascii for Raster format => raster.gridwrite.format("arcgis","filename.txt")
#Kri=raster.getColumn("Migrate.spde.Thickness.kriging")
#len(Kri)
#KriG= np.array(Kri)
#KriG = np.where(KriG > 1000, -999999, KriG)
#KriG= np.where(KriG==0, -999999, KriG) 
#np.savetxt("filename.txt", KriG)